In [1]:
import os,sys,torch
import pandas as pd

: 

: 

In [3]:
from bulian.Tabular.synthesizers import TwinSynthesizer,PrivateTwinSynthesizer
from bulian.metrics import *
from bulian.metrics.reports import *
from bulian.metrics import compute_metrics
from bulian.metrics.single_table import SingleTableMetric
from bulian.metrics.single_table import *

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [6]:
import torch
torch.__version__

'1.12.0'

In [7]:
torch.cuda.is_available()

True

In [8]:
!nvidia-smi

Fri Jul 29 14:04:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   53C    P8    26W / 370W |    854MiB / 24576MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###### 

## Normal API: Non-privately differentiable synthesizer

In [9]:
data = pd.read_csv("adult.csv")

In [10]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [11]:
synth = TwinSynthesizer(batch_size=200,device='cpu')   ### cpu else cuda

In [12]:
synth.fit(data=data,epochs=20,discrete_columns=discrete_columns)

Epoch: [0]  [  0/161]  eta: 0:00:08  loss_g: 2.0207 (2.0207)  loss_d: 0.0142 (0.0142)  loss: 2.0349 (2.0349)  time: 0.0532  data: 0.0000  max mem: 0
Epoch: [0]  [ 50/161]  eta: 0:00:05  loss_g: 1.3944 (1.7278)  loss_d: -0.6418 (-0.6082)  loss: 0.8024 (1.1196)  time: 0.0492  data: 0.0000  max mem: 0
Epoch: [0]  [100/161]  eta: 0:00:03  loss_g: 1.2429 (1.5635)  loss_d: 0.0401 (-0.4325)  loss: 1.2823 (1.1310)  time: 0.0498  data: 0.0000  max mem: 0
Epoch: [0]  [150/161]  eta: 0:00:00  loss_g: 0.7695 (1.3572)  loss_d: 0.2099 (-0.2477)  loss: 0.9733 (1.1095)  time: 0.0506  data: 0.0000  max mem: 0
Epoch: [0]  [161/161]  eta: 0:00:00  loss_g: 0.9207 (1.3282)  loss_d: 0.1827 (-0.2210)  loss: 1.0504 (1.1073)  time: 0.0499  data: 0.0000  max mem: 0
Epoch: [0] Total time: 0:00:08
Epoch: [1]  [  0/161]  eta: 0:00:07  loss_g: 1.0147 (1.0147)  loss_d: -0.0040 (-0.0040)  loss: 1.0107 (1.0107)  time: 0.0458  data: 0.0000  max mem: 0
Epoch: [1]  [ 50/161]  eta: 0:00:05  loss_g: 0.5482 (0.6983)  loss_d

In [13]:
sample = synth.sample(1000)

In [14]:
sample

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,20,?,283339,HS-grad,10,Never-married,Sales,Not-in-family,White,Female,59,0,15,United-States,<=50K
1,54,Local-gov,132887,9th,10,Never-married,Farming-fishing,Own-child,White,Female,-12,0,15,United-States,<=50K
2,44,Self-emp-not-inc,415897,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,16185,1,51,United-States,>50K
3,46,Local-gov,217836,Some-college,10,Never-married,Machine-op-inspct,Husband,White,Male,21,-4,40,Cuba,<=50K
4,24,Private,133365,Assoc-acdm,10,Never-married,Handlers-cleaners,Not-in-family,White,Female,-71,-1,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,22,Private,315985,Bachelors,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,-1,-1,40,United-States,<=50K
996,35,Private,186220,Some-college,11,Never-married,Other-service,Not-in-family,White,Female,-54,-3,16,Germany,<=50K
997,30,Private,47154,11th,10,Widowed,Craft-repair,Not-in-family,Other,Male,-27,-2,23,United-States,<=50K
998,50,Local-gov,382909,HS-grad,9,Married-civ-spouse,Tech-support,Husband,White,Male,-66,-1,49,United-States,<=50K


In [15]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

In [16]:
metrics

{'BNLogLikelihood': bulian.metrics.single_table.bayesian_network.BNLogLikelihood,
 'LogisticDetection': bulian.metrics.single_table.detection.sklearn.LogisticDetection,
 'SVCDetection': bulian.metrics.single_table.detection.sklearn.SVCDetection,
 'BinaryDecisionTreeClassifier': bulian.metrics.single_table.efficacy.binary.BinaryDecisionTreeClassifier,
 'BinaryAdaBoostClassifier': bulian.metrics.single_table.efficacy.binary.BinaryAdaBoostClassifier,
 'BinaryLogisticRegression': bulian.metrics.single_table.efficacy.binary.BinaryLogisticRegression,
 'BinaryMLPClassifier': bulian.metrics.single_table.efficacy.binary.BinaryMLPClassifier,
 'MulticlassDecisionTreeClassifier': bulian.metrics.single_table.efficacy.multiclass.MulticlassDecisionTreeClassifier,
 'MulticlassMLPClassifier': bulian.metrics.single_table.efficacy.multiclass.MulticlassMLPClassifier,
 'LinearRegression': bulian.metrics.single_table.efficacy.regression.LinearRegression,
 'MLPRegressor': bulian.metrics.single_table.efficacy

In [17]:
compute_metrics(metrics,data, sample)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,MetricType,error
0,BNLogLikelihood,BayesianNetwork Log Likelihood,-1.378710e+01,0.000001,-inf,0.0,MAXIMIZE,Likelihood Metric,None
1,LogisticDetection,LogisticRegression Detection,6.802358e-01,0.680236,0.0,1.0,MAXIMIZE,Real vs Synthetic Dectection Metric,None
2,SVCDetection,SVC Detection,7.875496e-01,0.787550,0.0,1.0,MAXIMIZE,Real vs Synthetic Dectection Metric,None
3,BinaryDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...
4,BinaryAdaBoostClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...
5,BinaryLogisticRegression,None,NaN,NaN,0.0,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...
6,BinaryMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...
7,MulticlassDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...
8,MulticlassMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...
9,LinearRegression,None,NaN,NaN,-inf,1.0,MAXIMIZE,ML Efficacy Metric: R-Sq or F1,`target` must be passed either directly or ins...


#### Report with privacy metrics

In [18]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2022 14:18:21] "GET /_alive_473bb191-a133-4208-ac73-91bac858b93a HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8050/


#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('NormalAPI.pth')

#### 

## Adversarial API: Non-privately differentiable synthesizer

In [ ]:
synth = TwinSynthesizer(batch_size=200,device='cpu')   ### cpu else cuda

In [ ]:
synth.fit_adversarial(data=data,epochs=2,discrete_columns=discrete_columns,test_pct=0.33)

In [ ]:
sample = synth.sample_adversarial(data,1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('F:/Users/Kaggle/wids/AdversarialAPI.pth')

#### 

## Normal API: Privately differentiable synthesizer

In [ ]:
synth = PrivateTwinSynthesizer(epsilon=0.1,batch_size=64,device='cpu')   ### cpu else cuda

In [ ]:
synth.fit(data=data,discrete_columns=discrete_columns,update_epsilon=1)

In [ ]:
sample = synth.sample(1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

#### Save model to disk

In [ ]:
synth.save('PrivateModelNormalAPI.pth')

#### 

## Adversarial API: Privately differentiable synthesizer

In [ ]:
synth = PrivateTwinSynthesizer(epsilon=0.1,batch_size=64,device='cpu')   ### cpu else cuda

In [ ]:
synth.fit_adversarial(data=data,discrete_columns=discrete_columns,update_epsilon=1)

In [ ]:
sample = synth.sample_adversarial(data,1000)

In [ ]:
metrics = SingleTableMetric.get_subclasses()
numeric_features = ['capital-gain','capital-loss','hours-per-week']
discrete_columns = [
    "workclass",
    "education",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "native-country",
    "income",
]

#### Report with privacy metrics

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features, key_fields=['age','workclass','education'],sensitive_fields = ['income'])

#### Report without privacy metrics, but includes ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features,target='income')

#### Report without privacy metrics and without ML efficacy stuff

In [ ]:
get_full_report(data, sample,discrete_columns,numeric_features)

##### Generate Report on a Dashboard

Reports can be viewed on a dashboard webpage by using the option boolean <b>show_dashboard</b> parameter on the <b>get_full_report</b> function. The dashboard is a Dash application which runs on a local server. You can also specify which port to use for the local server, by default the app would run on <b>8050</b>.

In [ ]:
get_full_report(
    data,
    sample,
    discrete_columns,
    numeric_features,
    key_fields=['age','workclass','education'],
    sensitive_fields = ['income'],
    show_dashboard=True,
    port=8050
)

#### Save model to disk

In [ ]:
synth.save('PrivateModelAdvAPI.pth')

#### 

### Fin ###